In [2]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
taxis_zones = pd.read_parquet('main uber/taxis_zones.parquet') # --> VER PATH!!!!

In [22]:
path = "Data/fhvhv_tripdata_reducido.parquet"

data = pd.read_parquet(path)

In [19]:
def procesar_datos(datos):
    # Aquí puedes agregar tu lógica de ETL para transformar los datos según tus necesidades
    # Por ejemplo, podrías utilizar pandas para manipular datos tabulares si es necesario
    # datos = mi_logica_etl(datos)
    
    # Lectura de datos scrapeados
    #yellow_taxis = pd.read_parquet(datos)
    uber = datos

    # Se entiende que los registros nulos se deben a que son viajes que no fueron tomados en zona de aeropuertos, se procede a rellenarlos con valor tasa '0.00'
    uber.fillna(0.00, inplace=True)

    # Se crea una lista con las columnas a sumar
    columnas_a_sumar = ['base_passenger_fare', 'tolls', 'bcf','sales_tax','congestion_surcharge','airport_fee','driver_pay']

    # Se crea una nueva columna 'total_fare' que contiene la suma de las columnas de costos
    uber['total_fare'] = uber[columnas_a_sumar].sum(axis=1)
    
    # Se crea una lista con las columnas a borrar
    columnasAborrar = ['base_passenger_fare', 'tolls', 'bcf','sales_tax','congestion_surcharge','airport_fee','driver_pay']

    # Se dropean las columnas enlistadas
    uber_sumado = uber.drop(columnasAborrar, axis=1)

    # Realizar la fusión (merge) basándote en las columnas LocationID, en este caso utilizando la localización de recogida
    uber_merge = pd.merge(uber_sumado, taxis_zones, left_on='PULocationID', right_on='LocationID', how='inner')

    # Renombre de columnas
    uber_merge.rename(columns={'Borough': 'PUBorough','Zone' : 'PUZone' }, inplace=True)

    # Se dropea la columna LocationID
    uber_merge = uber_merge.drop('LocationID', axis=1)

    # Se realiza el merge usando las columnas de LocationID, en este caso utilizando la localización de arribo
    uber_merge2 = pd.merge(uber_merge, taxis_zones, left_on='DOLocationID', right_on='LocationID', how='inner')

    # Se renombran las columnas necesarias
    uber_merge2.rename(columns={'Borough': 'DOBorough','Zone' : 'DOZone' }, inplace=True)

    # Se dropea la columna LocationID
    uber_merge2 = uber_merge2.drop('LocationID', axis=1)

    # Se elimina la columna que contiene la hora en que llego el conductor al lugar de recogida.
    # Podria eliminarse tambien la columna de request_datetime, ya que a los objetivos del proyecto no estaría aportando algo valioso.
    columnasAborrar = ['on_scene_datetime', 'request_datetime']

    uber = uber_merge2.drop(columnasAborrar, axis=1)

    # Lista de nombres de columnas que deseas convertir a tipo datetime
    columnas_a_convertir = ['pickup_datetime','dropoff_datetime']

    # Itera sobre cada columna y realiza la conversión a tipo datetime
    for columna in columnas_a_convertir:
        uber[columna] = pd.to_datetime(uber[columna], errors='coerce')

    orden = ['pickup_datetime','PULocationID','PUBorough','PUZone','dropoff_datetime','DOLocationID','DOBorough','DOZone','trip_miles','trip_time','tips','total_fare']

    # Reordena las columnas según el nuevo orden
    uber = uber[orden]

    # Se renombran las columnas necesarias
    uber.rename(columns={'pickup_datetime': 'PUDatetime','dropoff_datetime' : 'DODatetime' }, inplace=True)

    # Crea dos nuevas columnas: 'fecha' y 'hora' -- para Pick Up y Drop Off
    uber['PUDate'] = uber['PUDatetime'].dt.date
    uber['PUTime'] = uber['PUDatetime'].dt.time

    uber['DODate'] = uber['DODatetime'].dt.date
    uber['DOTime'] = uber['DODatetime'].dt.time

    columnasAborrar = ['PUDatetime', 'DODatetime']

    uber = uber.drop(columnasAborrar, axis=1)

    orden = ['PUDate','PUTime','PULocationID','PUBorough','PUZone','DODate','DOTime','DOLocationID','DOBorough','DOZone','trip_miles','trip_time','tips','total_fare']
    # Reordena las columnas según el nuevo orden
    uber = uber[orden]

    return uber

In [16]:
uber = data

# Se entiende que los registros nulos se deben a que son viajes que no fueron tomados en zona de aeropuertos, se procede a rellenarlos con valor tasa '0.00'
uber.fillna(0.00, inplace=True)

# Se crea una lista con las columnas a sumar
columnas_a_sumar = ['base_passenger_fare', 'tolls', 'bcf','sales_tax','congestion_surcharge','airport_fee','driver_pay']

# Se crea una nueva columna 'total_fare' que contiene la suma de las columnas de costos
uber['total_fare'] = uber[columnas_a_sumar].sum(axis=1)

# Se crea una lista con las columnas a borrar
columnasAborrar = ['base_passenger_fare', 'tolls', 'bcf','sales_tax','congestion_surcharge','airport_fee','driver_pay']

# Se dropean las columnas enlistadas
uber_sumado = uber.drop(columnasAborrar, axis=1)

# Realizar la fusión (merge) basándote en las columnas LocationID, en este caso utilizando la localización de recogida
uber_merge = pd.merge(uber_sumado, taxis_zones, left_on='PULocationID', right_on='LocationID', how='inner')

# Renombre de columnas
uber_merge.rename(columns={'Borough': 'PUBorough','Zone' : 'PUZone' }, inplace=True)

# Se dropea la columna LocationID
uber_merge = uber_merge.drop('LocationID', axis=1)

# Se realiza el merge usando las columnas de LocationID, en este caso utilizando la localización de arribo
uber_merge2 = pd.merge(uber_merge, taxis_zones, left_on='DOLocationID', right_on='LocationID', how='inner')

# Se renombran las columnas necesarias
uber_merge2.rename(columns={'Borough': 'DOBorough','Zone' : 'DOZone' }, inplace=True)

# Se dropea la columna LocationID
uber_merge2 = uber_merge2.drop('LocationID', axis=1)

# Se elimina la columna que contiene la hora en que llego el conductor al lugar de recogida.
# Podria eliminarse tambien la columna de request_datetime, ya que a los objetivos del proyecto no estaría aportando algo valioso.
columnasAborrar = ['on_scene_datetime', 'request_datetime']

uber = uber_merge2.drop(columnasAborrar, axis=1)

# Lista de nombres de columnas que deseas convertir a tipo datetime
columnas_a_convertir = ['pickup_datetime','dropoff_datetime']

# Itera sobre cada columna y realiza la conversión a tipo datetime
for columna in columnas_a_convertir:
    uber[columna] = pd.to_datetime(uber[columna], errors='coerce')

orden = ['pickup_datetime','PULocationID','PUBorough','PUZone','dropoff_datetime','DOLocationID','DOBorough','DOZone','trip_miles','trip_time','tips','total_fare']

# Reordena las columnas según el nuevo orden
uber = uber[orden]

# Se renombran las columnas necesarias
uber.rename(columns={'pickup_datetime': 'PUDatetime','dropoff_datetime' : 'DODatetime' }, inplace=True)

# Crea dos nuevas columnas: 'fecha' y 'hora' -- para Pick Up y Drop Off
uber['PUDate'] = uber['PUDatetime'].dt.date
uber['PUTime'] = uber['PUDatetime'].dt.time

uber['DODate'] = uber['DODatetime'].dt.date
uber['DOTime'] = uber['DODatetime'].dt.time

columnasAborrar = ['PUDatetime', 'DODatetime']

uber = uber.drop(columnasAborrar, axis=1)

orden = ['PUDate','PUTime','PULocationID','PUBorough','PUZone','DODate','DOTime','DOLocationID','DOBorough','DOZone','trip_miles','trip_time','tips','total_fare']
# Reordena las columnas según el nuevo orden
uber = uber[orden]

In [18]:
uber.columns

Index(['PUDate', 'PUTime', 'PULocationID', 'PUBorough', 'PUZone', 'DODate',
       'DOTime', 'DOLocationID', 'DOBorough', 'DOZone', 'trip_miles',
       'trip_time', 'tips', 'total_fare'],
      dtype='object')

In [23]:
uber2 = procesar_datos(data)

In [24]:
uber2.head()

,PUDate,PUTime,PULocationID,PUBorough,PUZone,DODate,DOTime,DOLocationID,DOBorough,DOZone,trip_miles,trip_time,tips,total_fare
0,2023-01-24,15:59:23,136,Bronx,Kingsbridge Heights,2023-01-24,16:32:50,75,Manhattan,East Harlem South,6.520,2007,0.0,47.05
1,2023-01-26,18:59:17,136,Bronx,Kingsbridge Heights,2023-01-26,19:27:03,75,Manhattan,East Harlem South,8.120,1666,0.0,60.11
2,2023-01-18,15:36:27,136,Bronx,Kingsbridge Heights,2023-01-18,16:06:03,75,Manhattan,East Harlem South,7.240,1776,0.0,51.42
3,2023-01-04,08:08:48,136,Bronx,Kingsbridge Heights,2023-01-04,08:46:26,75,Manhattan,East Harlem South,6.803,2258,0.0,64.83
4,2023-01-27,04:12:55,136,Bronx,Kingsbridge Heights,2023-01-27,04:28:42,75,Manhattan,East Harlem South,7.580,947,0.0,36.73
